<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/MPI_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 18 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich-dev amd64 4.0-3 [7,375 kB]
Fetched 14.2 MB in 

In [8]:
%%writefile mpi_sum.c

#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>


int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int N = 1000; // Tamaño del arreglo
    int local_N = N / size; // Tamaño del subarreglo para cada proceso
    int local_sum = 0;
    int global_sum = 0;

    int* data = NULL;
    int* local_data = (int*)malloc(local_N * sizeof(int));

    // Inicializar el arreglo de datos en el proceso 0
    if (rank == 0) {
        data = (int*)malloc(N * sizeof(int));
        for (int i = 0; i < N; i++) {
            data[i] = i;
        }
    }

    // Distribuir los datos entre los procesos
    MPI_Scatter(data, local_N, MPI_INT, local_data, local_N, MPI_INT, 0, MPI_COMM_WORLD);

    // Calcular la suma local
    for (int i = 0; i < local_N; i++) {
        local_sum += local_data[i];
    }
    printf("La suma local es: %d proceso=%d\n", local_sum,rank);

    // Sumar las sumas locales para obtener la suma global
    MPI_Reduce(&local_sum, &global_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    // El proceso 0 imprime el resultado
    if (rank == 0) {
        printf("La suma global es: %d\n", global_sum);
    }

    MPI_Finalize();

    return 0;
}


Overwriting mpi_sum.c


In [9]:
! mpicc -o mpi_sum mpi_sum.c

In [12]:
! mpirun --oversubscribe --allow-run-as-root -np 10 ./mpi_sum

La suma local es: 24950 proceso=2
La suma local es: 34950 proceso=3
La suma local es: 14950 proceso=1
La suma local es: 4950 proceso=0
La suma local es: 74950 proceso=7
La suma local es: 54950 proceso=5
La suma local es: 44950 proceso=4
La suma local es: 64950 proceso=6
La suma local es: 84950 proceso=8
La suma local es: 94950 proceso=9
La suma global es: 499500


In [ ]:
%%writefile mpi_pi.c

// MPI for calculating PI

#include <stdio.h>
#include "mpi.h"
#include <math.h>

int main(int argc, char *argv[])
{
  int n, myid, numprocs, i;
  double PI25DT = 3.141592653589793238462643;
  double mypi, pi=0, h, sum, x;
  MPI_Init(NULL,NULL);
  MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
  MPI_Comm_rank(MPI_COMM_WORLD, &myid);

  if(myid==0){
    n=10000;
  }
  //comunicamos el numero de intervalos a cada proceso del mundo
  //calculamos los intervalos y repetimos si es necesario
     printf("my id : %d  mypi : %.16f, n=%d\n", myid,mypi, n);
    MPI_Bcast(&n, 1, MPI_INT, 0, MPI_COMM_WORLD);
      printf("after my id : %d  mypi : %.16f, n=%d\n", myid,mypi, n);
      h   = 1.0 / (double) n;
      sum = 0.0;
      for (i = myid + 1; i <= n; i += numprocs) {
            x = h * ((double)i - 0.5);
            sum += (4.0 / (1.0 + x*x));
      }
      mypi = h * sum;
      printf("my id : %d  mypi : %.16f, n=%d\n", myid,mypi, n);

      //colectamos los calculos de cada trabajador usando MPI_Reduce

      MPI_Reduce(&mypi, &pi, 1, MPI_DOUBLE, MPI_SUM, 3,
                MPI_COMM_WORLD);
      //imprimimos el valor luego de sumar
      if (myid == 3)
          printf("pi es aproximadamente %.16f, El error es %.16f\n",
                pi, fabs(pi - PI25DT));

  MPI_Finalize();
  return 0;
}

Overwriting mpi_pi.c


In [ ]:
! mpicc -o mpi_pi mpi_pi.c

In [ ]:
! mpirun --allow-run-as-root -np 4 ./mpi_pi

## Matrix multiplication

In [ ]:
%%writefile mpi_matrix_mult.c

/*
 * Un programa simple para multiplicar matrices
 * (Matrix_A  X  Matrix_B) => Matrix_C
 */

#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>
#include <time.h>


#define ARRAY_SIZE 10

typedef int matrix_t[ARRAY_SIZE][ARRAY_SIZE];

matrix_t MA,MB,MC;

/*
Rutina para multiplicar una fila por una columna y colocar un elemento en
matriz resultante.
*/
void mult(int size,
	  int row,
	  int column,
    int rowl,
	  matrix_t MA,
	  matrix_t MB,
	  int *MC)
{
  int position;
  //int row_l=0;
  MC[rowl]= 0;
  for(position = 0; position < size; position++) {
     MC[rowl] = MC[rowl] +
     ( MA[row][position]  *  MB[position][column] ) ;

  }
}

//colocamos valores random 1-10 en las matrices
void inicializamos_matriz(int size,
                    matrix_t MX)
{
    int   row, column;
    srand(time(0));
    for(row = 0; row < size; row++) {
    for (column = 0; column < size; column++) {
      MX[row][column]=rand()%10;
    }
  }
}

void imprimir_matriz(int size,
                    matrix_t MX)
{
    int   row, column;
    for(row = 0; row < size; row ++) {
    for (column = 0; column < size; column++) {
      printf("%5d ",MX[row][column]);
    }
    printf("\n");
  }
}

void imprimir_matriz2(int sizer,int sizec,
                    int *MX)
{
    int   row, column,i=0;
    for(row = 0; row < sizer; row++) {
    for (column = 0; column < sizec; column++) {
      printf("%5d ",MX[i]);
      i++;
    }
    printf("\n");
  }
}


// inicializamos valores y calcula los resultados

int main(void)
{
  int      size, row, column;

  size = ARRAY_SIZE;

//puntero a la matriz resultante
int *final_matrix;
int num_worker, rank;
MPI_Init(NULL, NULL);
MPI_Comm_size(MPI_COMM_WORLD, &num_worker);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);

if(rank == 0){
  // inicializamos los valores de la MA
  inicializamos_matriz(size, MA);
  //inicializamos los valores de la MB
  inicializamos_matriz(size, MB);
  //imprimimos
  printf("La matriz A es;\n");
  imprimir_matriz(size,MA);
  printf("La matriz B es;\n");
  imprimir_matriz(size,MB);
  //reservamos la memoria para la matriz final
    final_matrix = (int *) malloc(sizeof(int*) * size*size);
}


MPI_Bcast(MA, size*size , MPI_INT, 0, MPI_COMM_WORLD);
MPI_Bcast(MB, size*size , MPI_INT, 0, MPI_COMM_WORLD);

//chequeamos si proceso 1 recibio la información
if(rank == 1){
  printf("id:%d La matriz A es;\n",rank);
  imprimir_matriz(size,MA);
  printf("id:%d La matriz B es;\n",rank);
  imprimir_matriz(size,MB);
}

// determinamos la fila de inicio y fin para la proceso trabajador
int startrow = rank * ( size / num_worker);
int endrow = ((rank + 1) * ( size / num_worker)) -1;
//calculamos las sub-matrices
int number_of_rows = size / num_worker;
int *result_matrix = (int *) malloc(sizeof(int*) * number_of_rows * size);
    //multiplicamos
    int rowl=0;
    for(row = startrow; row <= endrow; row++) {
     for (column = 0; column < size; column++) {
      mult(size, row, column,rowl, MA, MB, result_matrix);
      rowl++;
      }
    }
   // log information
   if(rank==1){
      printf("id:%d  startrow=%d, endrow=%d,work=%d;\n",rank,startrow,endrow,number_of_rows*size);
      imprimir_matriz2(number_of_rows,size,result_matrix);
   }


//recolectamos los resutlados de la matriz
    MPI_Gather(result_matrix, number_of_rows*size, MPI_INT,
           final_matrix, number_of_rows*size,  MPI_INT, 0, MPI_COMM_WORLD);

  //imprimimos la matriz luego de recolectar los resultados
  if(rank == 0){
  printf("La matriz resultante C es (MPI);\n");
  imprimir_matriz2(size,size,final_matrix);
  }

 MPI_Finalize();

  return 0;
}



Writing mpi_matrix_mult.c


In [ ]:
! mpicc -o mpi_matrix_mult mpi_matrix_mult.c

In [ ]:
! mpirun --allow-run-as-root -np 5 ./mpi_matrix_mult

## Enviado estructuras de datos

In [ ]:
%%writefile mpi_send_struct.c

/*
 * enviado una strucutura
 */

#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>
#include <time.h>

int main(int argc, char** argv) {
    int ProcRank, ProcNum, mTag = 0;
    struct { int x;
        int y;
        int z;
        } point;
    MPI_Datatype ptype;
    MPI_Status status;
    MPI_Init(NULL, NULL);
    MPI_Comm_rank(MPI_COMM_WORLD, &ProcRank);
    MPI_Type_contiguous(3, MPI_INT, &ptype);
    MPI_Type_commit(&ptype);
    if (ProcRank == 1) {
        point.x = 45; point.y = 36; point.z = 0;
        MPI_Send(&point, 1, ptype, 0, mTag, MPI_COMM_WORLD);
    }
    if (ProcRank == 0) {
        MPI_Recv(&point, 1, ptype, 1, mTag, MPI_COMM_WORLD, &status);
        printf("Proceso: %d recibio punto con  coordenadas (%d; %d; %d)\n", ProcRank, point.x, point.y, point.z);
    }
    MPI_Finalize();
}

Overwriting mpi_send_struct.c


In [ ]:
! mpicc -o mpi_send_struct mpi_send_struct.c

In [ ]:
! mpirun --allow-run-as-root -np 2 ./mpi_send_struct